In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

from selenium import webdriver

options = webdriver.ChromeOptions()
#options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")  # sandbox 주요 요소 처리불가-> nosandbox 처리 가능

#headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'}
#options.add_argument('headers')
driver = webdriver.Chrome(options=options)

In [44]:
url='https://huggingface.co/papers'
driver.get(url)
time.sleep(1)

# 전 날 페이지로 넘기기
# for d in range(2):

all_title=[]
all_news=[]
all_url=[]
all_author=[] 
all_stars=[]

    
titles=driver.find_elements(By.TAG_NAME,'h3')# 제목
stars=driver.find_elements(By.TAG_NAME,'a>div>div') # 좋아요

for i in titles:
    all_title.append(i.text)
all_title=[i for i in all_title if i]  # 빈칸제거

for i in stars:
    all_stars.append(i.text)


for n in all_title:

    print(n)
    driver.find_element(By.PARTIAL_LINK_TEXT,n).click() #페이지 들어가기
    time.sleep(1)
    
    nourl=driver.current_url#url 저장
    all_url.append(nourl)
    time.sleep(1)
    
    news=driver.find_element(By.TAG_NAME,'p') # 뉴스 본문 저장
    all_news.append(news.text)


    author=driver.find_element(By.CLASS_NAME,'whitespace-nowrap.underline') #저자 
    all_author.append(author.text)
        
    
    driver.back()

    # # 잠깐 쉬고 전 페이지 가기
    # time.sleep(3)
    # driver.find_element(By.CLASS_NAME,'btn.gap-2').click() # 사이트 전 날



FreeInit: Bridging Initialization Gap in Video Diffusion Models
FreeControl: Training-Free Spatial Control of Any Text-to-Image Diffusion Model with Any Condition
DiffMorpher: Unleashing the Capability of Diffusion Models for Image Morphing
Rethinking Compression: Reduced Order Modelling of Latent Features in Large Language Models
Alignment for Honesty
Interfacing Foundation Models' Embeddings
CCM: Adding Conditional Controls to Text-to-Image Consistency Models
Fast Training of Diffusion Transformer with Extreme Masking for 3D Point Clouds Generation
VILA: On Pre-training for Visual Language Models
Honeybee: Locality-enhanced Projector for Multimodal LLM
Steering Llama 2 via Contrastive Activation Addition
Llama Guard: LLM-based Input-Output Safeguard for Human-AI Conversations
COLMAP-Free 3D Gaussian Splatting
How Well Does GPT-4V(ision) Adapt to Distribution Shifts? A Preliminary Investigation
PEEKABOO: Interactive Video Generation via Masked-Diffusion
"I Want It That Way": Enabling 

In [45]:
sumnews=[]
for t,n,u,a,s in zip(all_title,all_news,all_url,all_author,all_stars):
    singles={'title':t,'news':n,'url':u,'author':a,'like':s}
    sumnews.append(singles)

import pandas as pd
news_df=pd.DataFrame(sumnews)
news_df

,title,news,url,author,like
0,FreeInit: Bridging Initialization Gap in Video...,Though diffusion-based video generation has wi...,https://huggingface.co/papers/2312.07537,Tianxing Wu,8
1,FreeControl: Training-Free Spatial Control of ...,Recent approaches such as ControlNet offer use...,https://huggingface.co/papers/2312.07536,Sicheng Mo,5
2,DiffMorpher: Unleashing the Capability of Diff...,Diffusion models have achieved remarkable imag...,https://huggingface.co/papers/2312.07409,Kaiwen Zhang,3
3,Rethinking Compression: Reduced Order Modellin...,Due to the substantial scale of Large Language...,https://huggingface.co/papers/2312.07046,Arnav Chavan,3
4,Alignment for Honesty,Recent research has made significant strides i...,https://huggingface.co/papers/2312.07000,Yuqing Yang,3
5,Interfacing Foundation Models' Embeddings,"We present FIND, a generalized interface for a...",https://huggingface.co/papers/2312.07532,Xueyan Zou,1
6,CCM: Adding Conditional Controls to Text-to-Im...,Consistency Models (CMs) have showed a promise...,https://huggingface.co/papers/2312.06971,Jie Xiao,1
7,Fast Training of Diffusion Transformer with Ex...,Diffusion Transformers have recently shown rem...,https://huggingface.co/papers/2312.07231,Shentong Mo,1
8,VILA: On Pre-training for Visual Language Models,Visual language models (VLMs) rapidly progress...,https://huggingface.co/papers/2312.07533,Ji Lin,1
9,Honeybee: Locality-enhanced Projector for Mult...,"In Multimodal Large Language Models (MLLMs), a...",https://huggingface.co/papers/2312.06742,Junbum Cha,1


In [47]:
news_df.to_excel('2023-12-13.xlsx')